In [1]:
#!pip install umap-learn umap-learn[plot] umap
#!pip install holoviews
#!pip install -U ipykernel
#!pip3 install keras-visualizer
#!pip install giotto-tda  

In [2]:
# reload modules before executing user code
%load_ext autoreload
# reload all modules every time before executing the Python code
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import gtda.diagrams as diag
from gtda.diagrams import Scaler, Filtering, PersistenceEntropy, BettiCurve, PairwiseDistance
from gtda.homology import VietorisRipsPersistence
import gtda.graphs as gr
from gtda.pipeline import Pipeline
from gtda.plotting import plot_point_cloud, plot_heatmap
from gtda.graphs import KNeighborsGraph, GraphGeodesicDistance
from gtda.mapper import (
    CubicalCover,
    OneDimensionalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph)
from gtda.mapper import Eccentricity, Entropy

import umap.umap_ as umap

from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff


## Builiding data

### PubChem Search

*References: Wang Y, Xiao J, Suzek TO, Zhang J, Wang J, Bryant SH. PubChem: a public information system for analyzing bioactivities of small molecules. Nucleic Acids Res. 2009 Jul;37(Web Server issue):W623-33. doi: 10.1093/nar/gkp456. Epub 2009 Jun 4. PMID: 19498078; PMCID: PMC2703903.*

In [4]:
def load_csv(path):
    return pd.read_csv(path, 
                       engine='c', 
                       parse_dates=True, 
                       infer_datetime_format=True, 
                       low_memory=False)

DF = load_csv('DDH Data with Properties.csv')
DF = DF.dropna()

In [5]:
def min_max(df):
    min_max_scaler = MinMaxScaler(feature_range=(0,1))
    df_scaled = min_max_scaler.fit_transform(df[[item for item in df.columns]])
    df_scaled= pd.DataFrame(df_scaled)
    df = df.mask(df==0).fillna(method='backfill')
    return df

In [6]:
df = DF.drop(columns=['CID', 	'SMILES', 	'MolecularFormula', 'InChI', 	'InChIKey', 	'IUPACName'])

In [7]:
""" pIC50 = 50% Log inhibitory concentration of the compound (in microM).
Higher values of pIC50 indicate exponentially more potent inhibitors"""

#train values
clean = df[df['pIC50'] != 'BLINDED']
clean['pIC50'] = clean['pIC50'].astype('float')
clean = clean.dropna()
clean = clean.reset_index(drop=True)

#test values
predict = df[df['pIC50'] == 'BLINDED'] 

## Deep Feed-Forward Neural Network


In [8]:
import keras
#from keras.utils import to_categorical
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential
from datetime import datetime
import tensorboard
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

In [9]:
dataset = clean
X = dataset.drop(columns='pIC50')
y = dataset['pIC50']
y = pd.cut(y, bins=[-3.0,  0.0, 1.5], labels=[0, 1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [10]:
input_dim = len(clean.drop(columns='pIC50').columns)  #floats

model = Sequential()
#The Dense function in Keras constructs a fully connected neural network layer, 
#  automatically initializing the weights as biases.
#First hidden layer
#The input layer is specified as a parameter 
#  to the first Dense object's constructor.
#Regularrizer tells Keras to include the squared values of those parameters in our overall 
#  loss function, and weight them by 0.01 in the loss function
model.add(Dense(50, activation='relu', kernel_initializer='random_normal',
                kernel_regularizer=regularizers.l2(0.05), input_dim=input_dim,
                ))
#model.add(Dropout(0.3))    
#Second hidden layer
model.add(Dense(40, activation='relu', kernel_initializer='random_normal',
                kernel_regularizer=regularizers.l2(0.05) ))

model.add(Dense(20, activation='relu', kernel_initializer='random_normal',
                kernel_regularizer=regularizers.l2(0.05) ))
#model.add(Dropout(0.3))   
#Output layer
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal',
                kernel_regularizer=regularizers.l2(0.05) ))
#model.add(Dropout(0.3))   
# compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1700      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 4,581
Trainable params: 4,581
Non-trainable params: 0
_________________________________________________________________


In [12]:
# fit your model to the training data.
history = model.fit(X_train, 
          y_train,
          epochs=100,
          batch_size=10,
          shuffle=True,
          verbose=2,
          validation_data=(X_test, y_test))

Epoch 1/100
6/6 - 15s - loss: 1.1903 - accuracy: 0.6500 - val_loss: 1.0778 - val_accuracy: 0.7097
Epoch 2/100
6/6 - 0s - loss: 0.8544 - accuracy: 0.9167 - val_loss: 1.0023 - val_accuracy: 0.7097
Epoch 3/100
6/6 - 0s - loss: 0.6654 - accuracy: 0.9167 - val_loss: 1.0791 - val_accuracy: 0.7097
Epoch 4/100
6/6 - 0s - loss: 0.5718 - accuracy: 0.9167 - val_loss: 1.1848 - val_accuracy: 0.7097
Epoch 5/100
6/6 - 0s - loss: 0.5410 - accuracy: 0.9167 - val_loss: 1.1626 - val_accuracy: 0.7097
Epoch 6/100
6/6 - 0s - loss: 0.5067 - accuracy: 0.9167 - val_loss: 1.0622 - val_accuracy: 0.7097
Epoch 7/100
6/6 - 0s - loss: 0.4637 - accuracy: 0.9167 - val_loss: 0.9806 - val_accuracy: 0.7097
Epoch 8/100
6/6 - 0s - loss: 0.4363 - accuracy: 0.9167 - val_loss: 0.9610 - val_accuracy: 0.7097
Epoch 9/100
6/6 - 0s - loss: 0.4145 - accuracy: 0.9167 - val_loss: 0.8900 - val_accuracy: 0.7097
Epoch 10/100
6/6 - 0s - loss: 0.3987 - accuracy: 0.9167 - val_loss: 0.8538 - val_accuracy: 0.7097
Epoch 11/100
6/6 - 0s - loss

In [13]:
import plotly.express as px
dH = pd.DataFrame(history.history)
fig = go.Figure()

fig.add_trace(go.Scatter(x=dH.index, y=dH['loss'],
                    mode='lines',
                    name='Training loss',
                    line_color='red'))
fig.add_trace(go.Scatter(x=dH.index, y=dH['val_loss'],
                    mode='lines',
                    name='Validation loss',
                    line_color='orange'))
fig.add_trace(go.Scatter(x=dH.index, y=dH['accuracy'],
                    mode='lines',
                    name='Training accuracy',
                    line_color='green'))
fig.add_trace(go.Scatter(x=dH.index, y=dH['val_accuracy'],
                    mode='lines',
                    name='Validation accuracy',
                    line_color='olive'))
fig.update_layout(
    title="Accuracy and Loss",
    xaxis_title="Number of epochs")
fig.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')

In [33]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#model fit
model_loss, model_accuracy = model.evaluate(
    X_test, 
    y_test, 
    verbose=2,
    callbacks=[tensorboard_callback])
print(f"Validation Loss= {model_loss: .2f}, ; Validation Accuracy: {model_accuracy*100:.2f}%, for binary classification")

1/1 - 0s - loss: 0.7533 - accuracy: 0.7097
Validation Loss=  0.75, ; Validation Accuracy: 70.97%, for binary classification


## Confussion Matrix

In [15]:
y_pred=model.predict(X_test)
y_pred =(y_pred > 0.5)

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[22  0]
 [ 9  0]]


## Tensorboard

In [17]:
#%load_ext tensorboard
#%tensorboard --logdir logs

## Visualizing weights

In [18]:
layers = model.layers
model.get_layer('dense').output[0:]

<KerasTensor: shape=(None, 50) dtype=float32 (created by layer 'tf.__operators__.getitem')>

In [19]:
from keras.models import Model
data = clean.drop(columns=['pIC50'])

layer_name_0 = 'dense'
layer_name_1 = 'dense_1'
layer_name_2 = 'dense_2'
layer_name_3 = 'dense_3'

intermediate_layer_model_0 = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name_0).output)
intermediate_layer_model_1 = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name_1).output)
intermediate_layer_model_2 = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name_2).output)
intermediate_layer_model_3 = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name_3).output)

intermediate_output_0 = intermediate_layer_model_0.predict(data)
intermediate_output_1 = intermediate_layer_model_1.predict(data)
intermediate_output_2 = intermediate_layer_model_2.predict(data)
intermediate_output_3 = intermediate_layer_model_3.predict(data)

In [20]:
from gtda.homology import VietorisRipsPersistence

""" Connectivity information
0-dimensional homology β0 or H0, measures clusters; 
1-dimensional homology1 β1 or H1, measures loops; and
2- dimensional homology β2 or H2, measures voids (empty spaces) """
homology_dimensions = [0, 1, 2]
homology_dimensions = [0, 1, 2]
VR = VietorisRipsPersistence(
                        homology_dimensions=homology_dimensions,
                        coeff=3,
                        n_jobs=-1)

diagram_raw =VR.fit_transform(np.array(data)[None, : , :])
diagram_0 =VR.fit_transform(np.array(intermediate_output_0)[None, : , :])

In [21]:
#plot persistence diagram raw data
VR.fit_transform_plot(np.array(data).reshape(1, *np.array(data).shape))

array([[[  0.        ,   3.29765058,   0.        ],
        [  0.        ,   4.84565783,   0.        ],
        [  0.        ,   5.51750326,   0.        ],
        [  0.        ,   6.00432348,   0.        ],
        [  0.        ,   6.3010745 ,   0.        ],
        [  0.        ,   6.51422596,   0.        ],
        [  0.        ,   6.5406332 ,   0.        ],
        [  0.        ,   7.79481745,   0.        ],
        [  0.        ,   8.37479782,   0.        ],
        [  0.        ,   8.57462502,   0.        ],
        [  0.        ,  12.50482273,   0.        ],
        [  0.        ,  14.97803974,   0.        ],
        [  0.        ,  17.78689766,   0.        ],
        [  0.        ,  18.74373245,   0.        ],
        [  0.        ,  21.45097923,   0.        ],
        [  0.        ,  23.699049  ,   0.        ],
        [  0.        ,  25.6869545 ,   0.        ],
        [  0.        ,  25.94498444,   0.        ],
        [  0.        ,  27.55833817,   0.        ],
        [  0

In [22]:
#plot persistence diagram 
VR.fit_transform_plot(np.array(intermediate_output_1).reshape(1, *np.array(intermediate_output_1).shape))

array([[[0.        , 0.06822318, 0.        ],
        [0.        , 0.08363537, 0.        ],
        [0.        , 0.08766717, 0.        ],
        [0.        , 0.11250205, 0.        ],
        [0.        , 0.11520229, 0.        ],
        [0.        , 0.11846819, 0.        ],
        [0.        , 0.12466089, 0.        ],
        [0.        , 0.12950616, 0.        ],
        [0.        , 0.13271537, 0.        ],
        [0.        , 0.13716282, 0.        ],
        [0.        , 0.18226068, 0.        ],
        [0.        , 0.1901373 , 0.        ],
        [0.        , 0.24574885, 0.        ],
        [0.        , 0.25341496, 0.        ],
        [0.        , 0.25580463, 0.        ],
        [0.        , 0.26434824, 0.        ],
        [0.        , 0.27189627, 0.        ],
        [0.        , 0.29298261, 0.        ],
        [0.        , 0.29506561, 0.        ],
        [0.        , 0.31587851, 0.        ],
        [0.        , 0.33667788, 0.        ],
        [0.        , 0.33698827, 0

## Mapper algorithm

In [23]:
""" 1. Define filter function – can be any scikit-learn transformer
 It is returning a selection of columns of the data """
#filter_func = Projection() #columns=[0,1]
#filter_func = Entropy() 
filter_func = Eccentricity(metric= 'euclidean') #Eccentricities of points in a point cloud or abstract metric space.
""" The distance function (metric) can be: ‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, 
‘correlation’, ‘cosine’, ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘jensenshannon’, 
‘kulsinski’, ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, 
‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’."""
""" 2. Define cover """
cover = CubicalCover(n_intervals=20,
                     overlap_frac=0.5)
""" 3. Choose clustering algorithm – default is DBSCAN """
clusterer = DBSCAN(eps=8, 
                   min_samples=2, 
                   metric='euclidean')

""" 4. Initialise pipeline """
pipe_mapper = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=-1)

In [24]:
plotly_params = {"node_trace": {"marker_colorscale": "RdBu"}}
fig = plot_static_mapper_graph(pipe_mapper,
                               X, 
                               layout='fruchterman_reingold',
                               #color_by_columns_dropdown=True,
                               color_variable =clean['pIC50'],
                               node_scale =20,
                               plotly_params=plotly_params)
fig.show(config={'scrollZoom': True})

In [25]:
plotly_params = {"node_trace": {"marker_colorscale": "RdBu"}}
fig = plot_static_mapper_graph(pipe_mapper,
                               intermediate_output_0, 
                               layout='fruchterman_reingold',
                               #color_by_columns_dropdown=True,
                               color_variable =clean['pIC50'],
                               node_scale =20,
                               plotly_params=plotly_params)
fig.show(config={'scrollZoom': True})

In [26]:
plotly_params = {"node_trace": {"marker_colorscale": "RdBu"}}
fig = plot_static_mapper_graph(pipe_mapper,
                               intermediate_output_1, 
                               layout='fruchterman_reingold',
                               #color_by_columns_dropdown=True,
                               color_variable =clean['pIC50'],
                               node_scale =20,
                               plotly_params=plotly_params)
fig.show(config={'scrollZoom': True})

In [27]:
plotly_params = {"node_trace": {"marker_colorscale": "RdBu"}}
fig = plot_static_mapper_graph(pipe_mapper,
                               intermediate_output_2, 
                               layout='fruchterman_reingold',
                               #color_by_columns_dropdown=True,
                               color_variable =clean['pIC50'],
                               node_scale =20,
                               plotly_params=plotly_params)
fig.show(config={'scrollZoom': True})

In [28]:
plotly_params = {"node_trace": {"marker_colorscale": "RdBu"}}
fig = plot_static_mapper_graph(pipe_mapper,
                               intermediate_output_3, 
                               layout='fruchterman_reingold',
                               #color_by_columns_dropdown=True,
                               color_variable =clean['pIC50'],
                               node_scale =20,
                               plotly_params=plotly_params)
fig.show(config={'scrollZoom': True})

In [31]:
""" Uniform Manifold Approximation and Projection 
for Dimensionality Reduction -UMAP"""
#UMAP class
n_neighbors=10
min_dist=0.5 
"""default 0.1 (smaller values will 
result in a more clustered/clumped embedding)"""
umap_2d = umap.UMAP(n_neighbors=n_neighbors,
                    #metric=metric,
                    n_components=2, #n_components = dimensions
                    min_dist=min_dist,
                    init='random', 
                    random_state=0) 

dfData = pd.DataFrame(umap_2d.fit_transform(X))
dfL1 = pd.DataFrame(umap_2d.fit_transform(intermediate_output_0))
dfL2 = pd.DataFrame(umap_2d.fit_transform(intermediate_output_1))
dfL3 = pd.DataFrame(umap_2d.fit_transform(intermediate_output_2))
dfL4 = pd.DataFrame(umap_2d.fit_transform(intermediate_output_3))

fig = make_subplots( rows=3, cols=2, 
                    subplot_titles=("Original Data", 
                                    "Layer 1", 
                                    "Layer 2", 
                                    "Layer 3",
                                    "Output layer",
                                    "Original Data")
)
fig.add_trace(go.Scatter(x=dfData[0], y=dfData[1], 
                         mode="markers", 
                         marker_color=pd.DataFrame(y)['pIC50'],
                         marker_size = 4), 
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfL1[0], y=dfL1[1], 
                         mode="markers", 
                         marker_color=pd.DataFrame(y)['pIC50'],
                         marker_size = 4),
              row=1, col=2)

fig.add_trace(go.Scatter(x=dfL2[0], y=dfL2[1],
                         mode="markers", 
                         marker_color=pd.DataFrame(y)['pIC50'],
                         marker_size = 4),
              row=2, col=1)

fig.add_trace(go.Scatter(x=dfL3[0], y=dfL3[1],
                         mode="markers", 
                         marker_color=pd.DataFrame(y)['pIC50'],
                         marker_size = 4),
              row=2, col=2)

fig.add_trace(go.Scatter(x=dfL4[0], y=dfL4[1],
                         mode="markers", 
                         marker_color=pd.DataFrame(y)['pIC50'],
                         marker_size = 4),
              row=3, col=1)

fig.add_trace(go.Scatter(x=dfData[0], y=dfData[1],
                         mode="markers", 
                         marker_color=pd.DataFrame(y)['pIC50'],
                         marker_size = 4),
              row=3, col=2)

fig.update_layout(height=900, 
                  width=800, 
                  showlegend=False,
                  title_text="Layers data projections")
fig.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')

fig.show()